In [1]:
import sys 
import numpy as np
from resource import * 
import time
import psutil
import os 

# Base Code Workings

## reading in file 

In [94]:
base_strings = []
index_1 = []
index_2 = []
string_counter=0
with open("../data/SampleTestCases/input1.txt","r") as f:
    for line in f.readlines():
        #remove all whitespaces in text file from reading in line by line 
        line=line.strip()
        # select only the base sequence to append to the sequence list 
        if not line.isnumeric():
            base_strings.append(line)
            #counter to make sure we are only appending strings not not the indices 
            string_counter+=1
        ##if the line is not a base sequence, then it is an index
        else:
            #if the string counter ==2, then that means are appending the second set of indices
            if string_counter == 2:
                index_2.append(int(line))
            #if the string !=2, then that means we are appending the first set of indices 
            else:
                index_1.append(int(line))
print(base_strings,index_1,index_2)
        




['ACTG', 'TACG'] [3, 6, 1, 1] [1, 2, 9, 2]


## generate strings

In [95]:
sequences=[]
for string in base_strings:
    #print(string)
    ##check if its string 1 or string 2 we are using 
    if string == base_strings[0]:
        for i in index_1:
            string = string[0:i+1] + string + string[i+1:]
        sequences.append(string)
    elif string == base_strings[1]:
        for i in index_2:
            string = string[0:i+1] + string + string[i+1:]
        sequences.append(string)
sequences
    

['ACACACTGACTACTGACTGGTGACTACTGACTGGACTGACTACTGACTGGTGACTACTGACTGG',
 'TATTATTATACGCTATTATACGCGACGCGGACGCGTATACGCTATTATACGCGACGCGGACGCG']

# Functions for Basic.Py

## reading file input

In [2]:
def input_read(file):
    base_strings = []
    index_1 = []
    index_2 = []
    j=0
    k=0
    string_counter=0
    with open(file,"r") as f:
        for line in f.readlines():
            #remove all whitespaces in text file from reading in line by line 
            line=line.strip()
            # select only the base sequence to append to the sequence list 
            if not line.isnumeric():
                base_strings.append(line)
                #counter to make sure we are only appending strings not not the indices 
                string_counter+=1
            ##if the line is not a base sequence, then it is an index
            else:
                #if the string counter ==2, then that means are appending the second set of indices
                if string_counter == 2:
                    index_2.append(int(line))
                    k+=1
                #if the string !=2, then that means we are appending the first set of indices 
                else:
                    index_1.append(int(line))
                    j+=1
    return base_strings,index_1,index_2,k,j

In [9]:
inputs=input_read("../data/SampleTestCases/input5.txt")
inputs

(['ACGT', 'TACG'],
 [3, 6, 1, 1, 5, 6, 7, 8, 9, 20],
 [1, 2, 0, 4, 3, 2, 0, 5, 6, 17],
 10,
 10)

## generating strings

In [4]:
def generate_sequences(base_strings,index_1,index_2,k,j):
    sequences=[]
    seq1=base_strings[0]
    seq2=base_strings[1]
    ##check if its string 1 or string 2 we are using 
    for i in index_1:
        seq1 = seq1[0:i+1] + seq1 + seq1[i+1:]
    if len(seq1) == (2**j)*len(base_strings[0]):
        sequences.append(seq1)
    for i in index_2:
        seq2 = seq2[0:i+1] + seq2 + seq2[i+1:]
    if len(seq2) == (2**k)*len(base_strings[1]):   
        sequences.append(seq2)
    return sequences

        

In [10]:
test=generate_sequences(inputs[0],inputs[1],inputs[2],inputs[3],inputs[4])
test

['ACACACAAAAACACACAAAACACACACAAAAACACACAAAACACACAAACACACAACACACACACACGTACGACGTACGTTGTACGACGTACGTTACGTACGACGTACGTTGTACGACGTACGTTGTACGACGTACGTTGTACGACGTACGTTACGTACGACGTACGTTGTACGACGTACGTTCACACGTACGACGTACGTTGTACGACGTACGTTACGTACGACGTACGTTGTACGACGTACGTTGTACGACGTACGTTGTACGACGTACGTTACGTACGACGTACGTTGTACGACGTACGTTCACACACACACGTACGACGTACGTTGTACGACGTACGTTACGTACGACGTACGTTGTACGACGTACGTTGTACGACGTACGTTGTACGACGTACGTTACGTACGACGTACGTTGTACGACGTACGTTCACACGTACGACGTACGTTGTACGACGTACGTTACGTACGACGTACGTTGTACGACGTACGTTGTACGACGTACGTTGTACGACGTACGTTACGTACGACGTACGTTGTACGACGTACGTTCACACAACACACACACACGTACGACGTACGTTGTACGACGTACGTTACGTACGACGTACGTTGTACGACGTACGTTGTACGACGTACGTTGTACGACGTACGTTACGTACGACGTACGTTGTACGACGTACGTTCACACGTACGACGTACGTTGTACGACGTACGTTACGTACGACGTACGTTGTACGACGTACGTTGTACGACGTACGTTGTACGACGTACGTTACGTACGACGTACGTTGTACGACGTACGTTCACACACACACGTACGACGTACGTTGTACGACGTACGTTACGTACGACGTACGTTGTACGACGTACGTTGTACGACGTACGTTGTACGACGTACGTTACGTACGACGTACGTTGTACGACGTACGTTCACACGTACGACGTACGTTGTACGACGTACGTTACGTACGACGTACGTTGTACGACGTACGTTG

## function for mismatch penalties 

In [6]:
def mismatch_penalty(base_1,base_2):
    nucs="ACGT"
    index_1=nucs.index(base_1)
    index_2=nucs.index(base_2)
    mismatch_matrix = [[0,110,48,94],
                        [110,0,118,48],
                        [48,118,0,110],
                        [94,48,110,0]]
    return mismatch_matrix[index_1][index_2]

## alignment 

In [7]:
def alignment(seq1,seq2):
    # set value of gaps 
    gap_pen=30
    # build dp table
    m=len(seq1)
    n=len(seq2)
    dp = np.zeros([m+1,n+1], dtype=int)
    #initialize dp table 
    dp[0:(m+1),0] = [i*gap_pen for i in range(m+1)]
    dp[0,0:(n+1)] = [j*gap_pen for j in range(n+1)]
    #fill dp table with penalty scores 
    for i in range(1,m+1):
        for j in range(1,n+1):
            # if the nucs are equal to each other 
            if seq1[i-1] == seq2[j-1]:
                dp[i][j] = dp[i-1][j-1]
            # if the nucs don't match each other, then find min of the potential penalties 
            else:
                dp[i][j] = min(dp[i-1][j-1] + mismatch_penalty(seq1[i-1],seq2[j-1]),
                                dp[i-1][j] + gap_pen,
                                dp[i][j-1] + gap_pen)
    
    # find optimal path from dp table 
    i= m
    j= n
    #tracing path 
    seq1_list=[]
    seq2_list=[]
    while i!=0 or j!=0:
        #case 1: seq2 has gap 
        if dp[i][j]==dp[i-1][j]+gap_pen:
            seq1_list.append(seq1[i-1])
            seq2_list.append('_')
            i-=1
        #case 2: seq1 has gap 
        elif dp[i][j]==dp[i][j-1]+gap_pen:
            seq1_list.append('_')
            seq2_list.append(seq2[j-1])
            j-=1

        #case 3: nucs match, so no penalty or nucs mismatch, but penalty is already accounted for so same path 
        else:
            seq1_list.append(seq1[i-1])
            seq2_list.append(seq2[j-1])
            i-=1
            j-=1


    s1_final=''
    s2_final=''
    for k in range(len(seq1_list)):
        s1_final += seq1_list.pop()
        s2_final += seq2_list.pop()
    wf = [s1_final, s2_final]
    return dp[m][n],wf

In [11]:
alignment(test[0],test[1])

(63996,
 ['___AC_____AC____AC___A______A_____A__A___AC___AC_ACAAAACAC_ACACAA__AAACACACAAAACACACAAACACACAACAC_AC_ACACACGTACGACGT__ACGTTGTACGACGTACG___TTACGTACGACGTACGTTG_TACGACGTACG___TTGTACGACGTACG__TTGTACGACGT_ACGT_TACGTACGACGTACG_TTGTACGACGTACGTTC_ACACGTACGACGTACG_TTGTACGACGTACG__TT_AC_GTACGACGTACG___TTGTACGACGTACGTTG_TACGACGTACG___TTGTACGACGTACGT_TACGTAC__GACGT__ACGTTGTACGACGTACGTTCACAC_ACACACGTACGACG_TACGTTGTACGACGTACGT_TACGT_ACGACGT__ACGTTGTACGACGTACG___TTGTACGACGTACGTTG_TACGACGTACG___TTACGTACGACGTACG__TTGTACGACGT_ACGTTCACACGTACGACGTACG_TTGTACGACGTACGT__TACGTACGACGTACG_TTGTACGACGTACG__TT_GT_A__CGACGTACG___TTGTACGACGTACGT__TACGTACGACGTACG_TTGTACGACGTACG___TTCAC___AC_A_ACAC_ACAC_AC_ACGTACGACGTACGTTG_TACGACGTACG___TTA__CGT_ACGACGT__ACGTTGTACGACGTACG___TTGTACGACGTACGTTG_TACGACGTACG___TTACGTACGACGTACG__TTGTACGACGT_ACGTTCACACGTACGACGTACG_TTGTACGACGTACGT__TACGTACGACGTACG_TTGTACGACGTACG__TT_GT_A__CGACGTACG___TTGTACGACGTACGT__TACGTACGACGTACG_TTGTACGACGTACG__T_TCAC__AC___AC_ACACGTACGACGTACG

# Memory and Time Functions

In [183]:
def process_memory():
    pid=os.getpid()
    process = psutil.Process(pid) 
    memory_info = process.memory_full_info() 
    memory_consumed = int(memory_info.rss/1024)
    return memory_consumed

In [169]:
process_memory()

257232

In [157]:
def time_wrapper(): 
    start_time = time.time() 
    call_algorithm() 
    end_time = time.time() 
    time_taken = (end_time - start_time)*1000
    return time_taken

# main function

In [181]:
def main(file):
    start_time = time.time() 
    start_memory=process_memory()
    inputs=input_read(file)
    sequences=generate_sequences(inputs[0],inputs[1],inputs[2],inputs[3],inputs[4])
    output=alignment(sequences[0],sequences[1])
    end_time = time.time()
    end_memory=process_memory()
    time_taken = (end_time - start_time)*1000
    memory_used=(end_memory - start_memory)
    return memory_used,time_taken,output

In [186]:
main("../data/SampleTestCases/input5.txt")

(220,
 135451.6143798828,
 (63996,
  ['___AC_____AC____AC___A______A_____A__A___AC___AC_ACAAAACAC_ACACAA__AAACACACAAAACACACAAACACACAACAC_AC_ACACACGTACGACGT__ACGTTGTACGACGTACG___TTACGTACGACGTACGTTG_TACGACGTACG___TTGTACGACGTACG__TTGTACGACGT_ACGT_TACGTACGACGTACG_TTGTACGACGTACGTTC_ACACGTACGACGTACG_TTGTACGACGTACG__TT_AC_GTACGACGTACG___TTGTACGACGTACGTTG_TACGACGTACG___TTGTACGACGTACGT_TACGTAC__GACGT__ACGTTGTACGACGTACGTTCACAC_ACACACGTACGACG_TACGTTGTACGACGTACGT_TACGT_ACGACGT__ACGTTGTACGACGTACG___TTGTACGACGTACGTTG_TACGACGTACG___TTACGTACGACGTACG__TTGTACGACGT_ACGTTCACACGTACGACGTACG_TTGTACGACGTACGT__TACGTACGACGTACG_TTGTACGACGTACG__TT_GT_A__CGACGTACG___TTGTACGACGTACGT__TACGTACGACGTACG_TTGTACGACGTACG___TTCAC___AC_A_ACAC_ACAC_AC_ACGTACGACGTACGTTG_TACGACGTACG___TTA__CGT_ACGACGT__ACGTTGTACGACGTACG___TTGTACGACGTACGTTG_TACGACGTACG___TTACGTACGACGTACG__TTGTACGACGT_ACGTTCACACGTACGACGTACG_TTGTACGACGTACGT__TACGTACGACGTACG_TTGTACGACGTACG__TT_GT_A__CGACGTACG___TTGTACGACGTACGT__TACGTACGACGTACG_TTGTACGACGTACG__T_TC

In [187]:
main("../data/SampleTestCases/input4.txt")

(0,
 270.2305316925049,
 (60,
  ['TCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCG_TCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGAA',
   'TCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGA_']))